In [13]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import time 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

2024-08-11 22:43:28.289646: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-11 22:43:28.477667: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-11 22:43:28.477711: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-08-11 22:43:29.159955: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [14]:


# 커스텀 BayesianLSTM 셀 구현
class BayesianLSTMCell(tf.keras.layers.LSTMCell):
    def __init__(self, units, **kwargs):
        super(BayesianLSTMCell, self).__init__(units, **kwargs)
        self.units = units
        self.kernel_posterior_fn = tfp.layers.default_mean_field_normal_fn()
        self.kernel_prior_fn = lambda dtype, shape, name, trainable, add_variable_fn: tfp.layers.default_multivariate_normal_fn(
            dtype=dtype, shape=shape, name=name, trainable=trainable, add_variable_fn=add_variable_fn)
        self.recurrent_kernel_posterior_fn = tfp.layers.default_mean_field_normal_fn()
        self.recurrent_kernel_prior_fn = lambda dtype, shape, name, trainable, add_variable_fn: tfp.layers.default_multivariate_normal_fn(
            dtype=dtype, shape=shape, name=name, trainable=trainable, add_variable_fn=add_variable_fn)

    def build(self, input_shape):
        self.kernel_posterior = self.kernel_posterior_fn(
            dtype=tf.float32,
            shape=[input_shape[-1], self.units * 4],
            name='kernel_posterior',
            trainable=True,
            add_variable_fn=self.add_weight
        )
        self.kernel_prior = self.kernel_prior_fn(
            dtype=tf.float32,
            shape=[input_shape[-1], self.units * 4],
            name='kernel_prior',
            trainable=True,
            add_variable_fn=self.add_weight
        )
        self.recurrent_kernel_posterior = self.recurrent_kernel_posterior_fn(
            dtype=tf.float32,
            shape=[self.units, self.units * 4],
            name='recurrent_kernel_posterior',
            trainable=True,
            add_variable_fn=self.add_weight
        )
        self.recurrent_kernel_prior = self.recurrent_kernel_prior_fn(
            dtype=tf.float32,
            shape=[self.units, self.units * 4],
            name='recurrent_kernel_prior',
            trainable=True,
            add_variable_fn=self.add_weight
        )
        super(BayesianLSTMCell, self).build(input_shape)

    def call(self, inputs, states, training=None):
        self.kernel = self.kernel_posterior.sample()
        self.recurrent_kernel = self.recurrent_kernel_posterior.sample()
        return super(BayesianLSTMCell, self).call(inputs, states, training=training)

def build_bayesian_lstm_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        tf.keras.layers.RNN(BayesianLSTMCell(10)),
        Dense(1)
    ])
    # model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    # return model
    kl_divergence = sum(model.losses)
    model.compile(optimizer=Adam(learning_rate=0.01),
                  loss=lambda y_true, 
                  y_pred: tf.keras.losses.mean_squared_error(y_true, y_pred) + kl_divergence)
    return model

In [15]:

def load_and_prepare_data(data_file):
    """
    주어진 CSV 파일을 불러와 기본 전처리를 수행.
    
    매개변수:
    - data_file: CSV 파일의 경로
    
    Returns:
    - data: 전처리된 데이터프레임
    """
    
    # Load the data
    data = pd.read_csv(data_file)

    # Prepare the data
    data['Time'] = pd.to_datetime(data['Time'])


    data['Target_MHC_Water_Level'] = data['MHC_Water_Level'].shift(-3)

    # Fill NaN values in 'Target_MHC_Water_Level' with the last value of 'MHC_Water_Level'
    data['Target_MHC_Water_Level'].fillna(data['MHC_Water_Level'].iloc[-1], inplace=True)
    
    
    
    # # Add lag variables for MHC_Water_Level from t-1 to t-10
    # for i in range(3, 11):
    #     data[f'MHC_Water_Level_lag_{i}'] = data['MHC_Water_Level'].shift(i)
    # data.dropna(inplace=True)
    
    
    #Add lag variables for MHC_Water_Level from t-1 to t-10
    for i in range(3, 6):
        data[f'MHC_Water_Level_lag_{i}'] = data['MHC_Water_Level'].shift(i)
        data[f'MH_Water_Level_lag_{i}'] = data['MH_Water_Level'].shift(i)
        data[f'PG_Water_Level_lag_{i}'] = data['PG_Water_Level'].shift(i)
        data[f'HH_Water_Level_lag_{i}'] = data['HH_Water_Level'].shift(i)
        data[f'GG_Water_Level_lag_{i}'] = data['GG_Water_Level'].shift(i)
    data.dropna(inplace=True)


    


    return data

def split_data(data):
    """
    데이터를 학습 및 테스트 데이터로 분할.
    
    매개변수:
    - data: 전체 데이터프레임
    
    Returns:
    - train_data: 학습 데이터
    - test_data: 테스트 데이터
    """
    # Split data
    train_size = int(len(data) * 0.8)
    train_data = data.iloc[:train_size]
    test_data = data.iloc[train_size:]
    
    return train_data, test_data

In [16]:


def bayes_prepare_train_test_sets(train_data, test_data):
    X_train = train_data.drop(columns=['Time', 'Target_MHC_Water_Level']).values
    y_train = train_data['Target_MHC_Water_Level'].values
    X_test = test_data.drop(columns=['Time', 'Target_MHC_Water_Level']).values
    y_test = test_data['Target_MHC_Water_Level'].values
    return X_train, y_train, X_test, y_test

In [17]:

                data = load_and_prepare_data('water_data.csv')
                train_data, test_data = split_data(data)
                X_train, y_train, X_test, y_test = bayes_prepare_train_test_sets(train_data, test_data)
                
                
                scaler_X = MinMaxScaler(feature_range=(0, 1))
                scaler_y = MinMaxScaler(feature_range=(0, 1))
                X_train_scaled = scaler_X.fit_transform(X_train)
                X_test_scaled = scaler_X.transform(X_test)
                y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
                
                model = build_bayesian_lstm_model(input_shape=(X_train_scaled.shape[1], 1))
                X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
                X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
                model.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=32, verbose=1)

2024-08-11 22:43:37.542912: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-11 22:43:37.542958: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2024-08-11 22:43:37.543271: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
501/501 [==============================] - 6s 9ms/step - loss: 0.0011
Epoch 2/100
501/501 [==============================] - 4s 8ms/step - loss: 6.1140e-04
Epoch 3/100
501/501 [==============================] - 4s 8ms/step - loss: 5.8809e-04
Epoch 4/100
501/501 [==============================] - 4s 8ms/step - loss: 5.7359e-04
Epoch 5/100
501/501 [==============================] - 4s 8ms/step - loss: 5.3064e-04
Epoch 6/100
501/501 [==============================] - 4s 8ms/step - loss: 4.8430e-04
Epoch 7/100
501/501 [==============================] - 4s 7ms/step - loss: 4.7703e-04
Epoch 8/100
501/501 [==============================] - 4s 8ms/step - loss: 3.9768e-04
Epoch 9/100
501/501 [==============================] - 27s 54ms/step - loss: 3.6517e-04
Epoch 10/100
501/501 [=====

In [8]:
bayes_pred_uncer =pd.read_csv('bayes_pred_uncer.csv')


In [69]:
def save_bayesian_visual_predictions_to_csv(model, 
                                            x,
                                            time_points, 
                                            scaler_y,
                                            n_iter=100):
    """
    예측 결과를 CSV 파일로 저장하는 함수
    
    매개변수:
    - model: 학습된 Bayesian LSTM 모델
    - x: 테스트 데이터의 특성 (정규화된 상태)
    - time_points: 시간대 포인트 (원래 시간 데이터)
    - scaler_y: 타겟 변수의 스케일러
    - n_iter: 예측 수행 횟수
    - filename: 저장할 CSV 파일 이름
    """

    
    # time_points = time_points[-3:]
    # print(time_points)
    
    predictions_scaled = np.zeros((len(time_points), n_iter))
    
    for j in range(n_iter):
        predictions_scaled[:, j] = model(x, training=True).numpy().flatten()
    
    # 예측 결과 역정규화
    predictions = scaler_y.inverse_transform(predictions_scaled)
    
    # 시간대와 예측 결과를 DataFrame으로 변환
    df_predictions = pd.DataFrame(
                                  predictions, 
                                  columns=[f'Prediction_{i+1}' for i in range(n_iter)])
    df_predictions.insert(0, 'Time', time_points)

    
    return df_predictions


In [71]:
import time


time_points = bayes_pred_uncer['Time']

# 시작 시간 기록
start_time = time.time()

# 실행할 코드
bayes_ppd_data = save_bayesian_visual_predictions_to_csv(model, 
                                                         X_test_scaled,
                                                         time_points, 
                                                         scaler_y,
                                                         n_iter=300)

# 종료 시간 기록
end_time = time.time()

# 경과 시간 계산
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time} seconds")


Execution time: 61.48241329193115 seconds


In [72]:
bayes_ppd_data = bayes_ppd_data.assign(Time=time_points.values)


In [83]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd

# @tf.function
def predict_and_save_bayesian_visual_predictions(model, 
                                                 x, 
                                                 time_points, 
                                                 scaler_y, 
                                                 n_iter=5000):
    """
    Bayesian LSTM 모델의 예측 결과를 여러 번 수행하여 불확실성과 신뢰 구간을 포함한
    예측 분포를 계산하고 이를 DataFrame으로 반환하는 함수.
    
    매개변수:
    - model: 학습된 Bayesian LSTM 모델
    - x: 테스트 데이터의 특성 (정규화된 상태)
    - time_points: 시간대 포인트 (원래 시간 데이터)
    - scaler_y: 타겟 변수의 스케일러
    - n_iter: 예측 수행 횟수
    
    반환값:
    - df_predictions: 예측 결과와 신뢰 구간을 포함한 DataFrame
    """

    # 여러 번의 예측 수행
    result = tf.TensorArray(tf.float32, size=n_iter)
    for i in tf.range(n_iter):
        result = result.write(i, model(x, training=True))
    result = result.stack()  # (n_iter, batch_size, output_size)
    
    # 평균 예측 값 계산
    prediction = tf.reduce_mean(result, axis=0)
    
    # 불확실성 계산 (표준편차)
    uncertainty = tf.math.reduce_std(result, axis=0)
    
    # 95% 신뢰구간 (credible interval) 계산
    lower_bound = tfp.stats.percentile(result, 2.5, axis=0)
    upper_bound = tfp.stats.percentile(result, 97.5, axis=0)
    
    # 예측 결과 역정규화
    prediction = scaler_y.inverse_transform(prediction.numpy())
    lower_bound = scaler_y.inverse_transform(lower_bound.numpy())
    upper_bound = scaler_y.inverse_transform(upper_bound.numpy())
    
    # 각 예측을 역정규화하고 DataFrame에 포함시키기 위해 loop
    predictions_scaled = np.zeros((len(time_points), n_iter))
    for j in range(n_iter):
        predictions_scaled[:, j] = scaler_y.inverse_transform(result[j].numpy().flatten().reshape(-1, 1)).flatten()

    # 시간대와 예측 결과를 DataFrame으로 변환
    df_predictions = pd.DataFrame(predictions_scaled, 
                                  columns=[f'Prediction_{i+1}' for i in range(n_iter)])
    df_predictions.insert(0, 'Time', time_points)
    df_predictions['Lower_Bound'] = lower_bound.flatten()
    df_predictions['Upper_Bound'] = upper_bound.flatten()
    df_predictions['Mean_Prediction'] = prediction.flatten()
    df_predictions['Uncertainty'] = uncertainty.numpy().flatten()

    
    # 열 순서 변경: Mean_Prediction, Uncertainty, Lower_Bound, Upper_Bound, Time + 나머지 Prediction 열
    cols = ['Time','Mean_Prediction', 'Uncertainty', 'Lower_Bound', 'Upper_Bound'] + [col for col in df_predictions.columns if col.startswith('Prediction_')]
    df_predictions = df_predictions[cols]
    
    return df_predictions


In [89]:
test_data

,Time,MHC_Water_Level,MH_Water_Level,PG_Water_Level,HH_Water_Level,GG_Water_Level,Target_MHC_Water_Level,MHC_Water_Level_lag_3,MH_Water_Level_lag_3,PG_Water_Level_lag_3,...,MHC_Water_Level_lag_4,MH_Water_Level_lag_4,PG_Water_Level_lag_4,HH_Water_Level_lag_4,GG_Water_Level_lag_4,MHC_Water_Level_lag_5,MH_Water_Level_lag_5,PG_Water_Level_lag_5,HH_Water_Level_lag_5,GG_Water_Level_lag_5
16012,2024-02-27 05:00:00,1.53,0.21,1.20,0.86,0.61,1.52,1.54,0.21,1.21,...,1.55,0.21,1.21,0.86,0.61,1.55,0.21,1.21,0.86,0.61
16013,2024-02-27 06:00:00,1.53,0.21,1.20,0.86,0.60,1.52,1.54,0.21,1.20,...,1.54,0.21,1.21,0.86,0.61,1.55,0.21,1.21,0.86,0.61
16014,2024-02-27 07:00:00,1.53,0.21,1.20,0.85,0.60,1.52,1.54,0.20,1.20,...,1.54,0.21,1.20,0.86,0.60,1.54,0.21,1.21,0.86,0.61
16015,2024-02-27 08:00:00,1.52,0.21,1.20,0.86,0.60,1.52,1.53,0.21,1.20,...,1.54,0.20,1.20,0.86,0.61,1.54,0.21,1.20,0.86,0.60
16016,2024-02-27 09:00:00,1.52,0.21,1.20,0.86,0.60,1.51,1.53,0.21,1.20,...,1.53,0.21,1.20,0.86,0.61,1.54,0.20,1.20,0.86,0.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20009,2024-08-11 18:00:00,1.50,0.27,0.96,0.72,0.33,1.51,1.51,0.28,0.96,...,1.51,0.27,0.96,0.72,0.35,1.51,0.27,0.96,0.72,0.35
20010,2024-08-11 19:00:00,1.50,0.28,0.95,0.72,0.32,1.52,1.50,0.28,0.96,...,1.51,0.28,0.96,0.72,0.34,1.51,0.27,0.96,0.72,0.35
20011,2024-08-11 20:00:00,1.50,0.28,0.96,0.71,0.32,1.52,1.50,0.28,0.96,...,1.50,0.28,0.96,0.72,0.34,1.51,0.28,0.96,0.72,0.34
20012,2024-08-11 21:00:00,1.51,0.28,0.96,0.71,0.32,1.52,1.50,0.27,0.96,...,1.50,0.28,0.96,0.72,0.33,1.50,0.28,0.96,0.72,0.34


In [84]:
# 예측 및 불확실성 계산, Bayesian PPD 데이터 생성
time_points = test_data["Time"] + pd.Timedelta(hours=3)
bayes_ppd_data = predict_and_save_bayesian_visual_predictions(model, 
                                                              X_test_scaled, 
                                                              time_points, 
                                                              scaler_y, 
                                                              n_iter=300)

# # 예측 데이터 정리 및 저장
# bayes_pred_uncer = pd.DataFrame({
#     'Time': time_points,
#     "True_Value": test_data['MHC_Water_Level'].shift(-3),
#     "Prediction": bayes_ppd_data['Mean_Prediction'],
#     "Uncertainty": bayes_ppd_data['Uncertainty']
# })

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/idhkdni/.local/lib/python3.7/site-packages/keras/backend.py", line 5134, in <genexpr>
    for ta, out in zip(output_ta_t, flat_output)  File "/home/idhkdni/.local/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py", line 245, in wrapped
    error_in_function=error_in_function)


In [87]:
bayes_ppd_data = bayes_ppd_data.assign(Time=time_points.values)

In [88]:
bayes_ppd_data

,Time,Mean_Prediction,Uncertainty,Lower_Bound,Upper_Bound,Prediction_1,Prediction_2,Prediction_3,Prediction_4,Prediction_5,...,Prediction_291,Prediction_292,Prediction_293,Prediction_294,Prediction_295,Prediction_296,Prediction_297,Prediction_298,Prediction_299,Prediction_300
0,2024-02-27 08:00:00,1.529848,0.000313,1.524414,1.536475,1.530097,1.531260,1.531292,1.532052,1.535637,...,1.526398,1.525872,1.526447,1.529583,1.524733,1.533506,1.530623,1.528365,1.534852,1.527605
1,2024-02-27 09:00:00,1.527591,0.000312,1.522243,1.534395,1.528022,1.528908,1.529112,1.529940,1.533231,...,1.524296,1.523654,1.524140,1.527266,1.522531,1.531011,1.528596,1.526105,1.532349,1.525399
2,2024-02-27 10:00:00,1.528251,0.000313,1.522778,1.534868,1.528468,1.529560,1.529662,1.530404,1.533936,...,1.524773,1.524363,1.524795,1.527935,1.523105,1.531783,1.529152,1.526741,1.533255,1.526024
3,2024-02-27 11:00:00,1.521580,0.000310,1.516202,1.527932,1.521722,1.523061,1.522950,1.523650,1.527427,...,1.518149,1.517701,1.518215,1.521376,1.516443,1.525444,1.522007,1.519973,1.526478,1.519360
4,2024-02-27 12:00:00,1.517414,0.000311,1.512114,1.524156,1.517777,1.518662,1.518806,1.519725,1.523106,...,1.514207,1.513492,1.513901,1.517012,1.512312,1.520894,1.518237,1.515909,1.522203,1.515155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3997,2024-08-11 21:00:00,1.524047,0.000290,1.518209,1.530091,1.527167,1.524237,1.528480,1.525862,1.526929,...,1.519464,1.524524,1.518945,1.526854,1.521866,1.524120,1.529194,1.520399,1.522116,1.523938
3998,2024-08-11 22:00:00,1.522455,0.000291,1.516585,1.528548,1.525589,1.522614,1.526886,1.524268,1.525291,...,1.517885,1.522948,1.517278,1.525252,1.520256,1.522501,1.527656,1.518770,1.520459,1.522354
3999,2024-08-11 23:00:00,1.523934,0.000290,1.518071,1.529950,1.527048,1.524103,1.528412,1.525716,1.526802,...,1.519365,1.524531,1.518868,1.526827,1.521818,1.523941,1.529086,1.520281,1.521852,1.523827
4000,2024-08-12 00:00:00,1.537170,0.000295,1.531135,1.543263,1.540396,1.537232,1.541773,1.538949,1.539937,...,1.532487,1.537935,1.532053,1.540133,1.535074,1.537135,1.542566,1.533548,1.535010,1.537190


In [59]:
a

,Time,Prediction_1,Prediction_2,Prediction_3,Prediction_4,Prediction_5,Prediction_6,Prediction_7,Prediction_8,Prediction_9,...,Prediction_291,Prediction_292,Prediction_293,Prediction_294,Prediction_295,Prediction_296,Prediction_297,Prediction_298,Prediction_299,Prediction_300
0,2024-08-11 22:00:00,1.526106,1.522710,1.523642,1.523435,1.523199,1.523363,1.520638,1.518787,1.527128,...,1.521467,1.521581,1.531349,1.521500,1.519252,1.521794,1.522144,1.521108,1.523878,1.523028
1,2024-08-11 23:00:00,1.539325,1.535879,1.536786,1.536638,1.536485,1.536480,1.533793,1.532016,1.540433,...,1.534840,1.534775,1.544807,1.534592,1.532485,1.535127,1.535393,1.534505,1.536927,1.536253
2,2024-08-12 00:00:00,1.552055,1.548527,1.549371,1.549372,1.549243,1.549096,1.546424,1.544828,1.553193,...,1.547707,1.547452,1.557702,1.547342,1.545184,1.547853,1.548098,1.547339,1.549518,1.549069


In [ ]:
# # CSV 파일로 저장
# bayes_pred_uncer.to_csv('../streamlit/data/bayes_pred_uncer.csv', index=False)
# bayes_ppd_data.to_csv('../streamlit/data/bayesian_ppd_visual.csv', index=False)

# print('Bayesian LSTM Model Learning & Prediction Finish\n')